# GENESIS Python Interface Hands-on Tutorial

This tutorial will guide you through using the Python interface for the analysis tools of GENESIS (Generalized Ensemble Simulation System). This interface allows you to easily use th analysis tools from Python scripts, integrate with other Python libraries, and visualize the results.

**This tutorial covers:**

1.  **Introduction**: Overview and setup.
2.  **Loading Molecular Information**: The `SMolecule` class and visualization with `py3Dmol`.
3.  **Loading & Visualizing Trajectory Data**: `crd_convert` and `py3Dmol`.
4.  **Basic Trajectory Analysis & Plotting**:
    * Distance, angle, and dihedral analysis (`trj_analysis`) with `plotly`.
    * Radius of gyration (`rg_analysis`) with `plotly`.
    * RMSD (`rmsd_analysis`) with `plotly`.
    * dRMSD (`drms_analysis`) with `plotly`.
    * Average structure (`avecrd_analysis`) with `py3Dmol`.
5.  **Advanced Analysis & Plotting**:
    * Mean square displacement and diffusion coefficient (`msd_analysis`, `diffusion_analysis`) with `plotly`.
6.  **Interfacing with External MD Libraries**: MDTraj and MDAnalysis.
7.  **Interfacing with Scikit-learn**: Feature preparation, t-SNE (dimensionality reduction), and DBSCAN (clustering) with `plotly`.
8.  **Interfacing with PyTorch**: Data preparation and a conformational autoencoder example with `plotly`.
9.  **Conclusion**

**Prerequisites**:

* GENESIS (with the Python interface correctly compiled and installed).
* Python 3.x
* NumPy (`pip install numpy`)
* Plotly (`pip install plotly`)
* py3Dmol (`pip install py3Dmol`)
* (Optional but Recommended for Section 6 & py3Dmol data prep) MDTraj (`pip install mdtraj`)
* (Optional but Recommended for Section 6) MDAnalysis (`pip install mdanalysis`)
* (Required for Section 7) Scikit-learn (`pip install scikit-learn`)
* (Required for Section 8) PyTorch (`pip install torch`)
* Jupyter Notebook or JupyterLab environment (highly recommended for running code blocks, especially `py3Dmol` and `plotly`).


**Input Files**:
This tutorial assumes you have the following files in your working directory:
* `BPTI_ionize.pdb`: A PDB file of your system (e.g., the initial structure for BPTI example).
* `BPTI_ionize.psf`: A PSF file corresponding to your PDB.
* `BPTI_run.dcd`: A DCD trajectory file from a GENESIS simulation of BPTI.


## 1. Introduction

The GENESIS Python interface bridges GENESIS's C/Fortran analysis routines with Python's scripting power. It simplifies workflows and enables integration with Python's scientific ecosystem.

Key components:
* `ctypes` wrappers (`libgenesis.py`, `s_molecule_c.py`, `s_trajectories_c.py`).
* Data converters (`py2c_util.py`, `c2py_util.py`).
* Pythonic classes (`SMolecule`, `STrajectories`).
* High-level analysis functions (`genesis_exe.py`).
* Control file utilities (`ctrl_files.py`).

Let's import modules. Ensure the GENESIS Python interface (`python_interface`) is in your `PYTHONPATH`.


In [ ]:
import os
import pathlib
import numpy as np
import warnings

# Plotting and visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import py3Dmol # For Jupyter environment

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning) 

# GENESIS Python interface modules
import s_molecule, s_trajectories, genesis_exe, ctrl_files
print("GENESIS Python interface modules imported successfully.")

# Define file paths (assuming they are in the current directory)
PDB_PATH = pathlib.Path("chignolin.pdb")
PSF_PATH = pathlib.Path("chignolin.psf")
DCD_PATH = pathlib.Path("chignolin.dcd")

## 2. Loading Molecular Information: `SMolecule` Class

The SMolecule class stores static molecular data (topology, atom properties). It's loaded from files like PDB and PSF.

In [ ]:
# Load molecular information
mol = s_molecule.SMolecule.from_file(pdb=PDB_PATH, psf=PSF_PATH)
print(f"Molecular information loaded from {PDB_PATH} and {PSF_PATH}.")
print(f"Number of atoms: {mol.num_atoms}")
print(f"First atom name: {''.join(mol.atom_name[0])}")
print(f"First atom coordinates: {mol.atom_coord[0]}")

## 3. Loading & Visualizing Trajectory Data: `STrajectories` Class

MD trajectory data (STrajectories objects) are typically loaded and processed using the crd_convert tool.

In [ ]:
# Define trajectory parameters
traj_params_list = [
    ctrl_files.TrajectoryParameters(
        trjfile=str(DCD_PATH), 
        md_step=22000, mdout_period=1, ana_period=1, repeat=1
    )
]

print(f"Load trajectory using crd_convert from: {DCD_PATH}")
trajs_array = genesis_exe.crd_convert(molecule=mol, 
    traj_params=traj_params_list, trj_format="DCD", #trj_natom=892,
    trj_type="COOR", selection_group=["all"], 
    fitting_method="TR+ROT", fitting_atom=1, pbc_correct="NO")
traj = trajs_array[0]
print(f"Trajectory loaded successfully via crd_convert.")
print(f"Frames: {traj.nframe}, Atoms: {traj.natom}")
print(f"Coordinates of 1st atom, 1st frame: {traj.coords[0, 0, :]}")

In [ ]:
# Visualize trajectory with py3Dmol (in Jupyter)
import mdtraj as md
import nglview as nv

mdtraj_top = mol.to_mdtraj_topology() 
mdtraj_traj = md.Trajectory(traj.coords / 10.0, mdtraj_top)

view = nv.show_mdtraj(mdtraj_traj)   # 直接渡すだけ
view.add_representation('licorice', color='sstruc')  # 任意で描画スタイル追加
view.center()
view

## 4. Basic Trajectory Analysis

### 4.1. Distance, Angle, Dihedral Analysis: `trj_analysis`

In [ ]:
distances_def = ["PROA:1:GLY:CA  PROA:2:TYR:CA"]
angles_def = ["PROA:1:GLY:CA  PROA:2:TYR:CA  PROA:3:ASP:CA"]
torsions_def = ["PROA:1:GLY:CA  PROA:2:TYR:CA  PROA:3:ASP:CA   PROA:4:PRO:CA"]

print(f"Using distance def: {distances_def}")
analysis_results = genesis_exe.trj_analysis(
    molecule=mol, trajs=traj, 
    distance=distances_def, 
    angle=angles_def, 
    torsion=torsions_def
)
time_axis = np.arange(traj.nframe)

In [ ]:
# Create subplots using plotly
fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=(f"Dist: {distances_def[0][:30]}...",
                                  f"Angle: {angles_def[0][:30]}...",
                                  f"Dihedral: {torsions_def[0][:30]}..."))

# Add traces
fig.add_trace(go.Scatter(x=time_axis, y=analysis_results.distance[:, 0], name="Distance"), row=1, col=1)
fig.add_trace(go.Scatter(x=time_axis, y=analysis_results.angle[:, 0], name="Angle"), row=1, col=2)
fig.add_trace(go.Scatter(x=time_axis, y=analysis_results.torsion[:, 0], name="Dihedral"), row=1, col=3)

# Update layout
fig.update_layout(height=500, width=1200, showlegend=True,
                 title_text="Trajectory Analysis Results")
fig.update_xaxes(title_text="Frame", row=1, col=1)
fig.update_xaxes(title_text="Frame", row=1, col=2)
fig.update_xaxes(title_text="Frame", row=1, col=3)
fig.update_yaxes(title_text="Distance (Å)", row=1, col=1)
fig.update_yaxes(title_text="Angle (°)", row=1, col=2)
fig.update_yaxes(title_text="Dihedral (°)", row=1, col=3)

fig.show()
print("Plotly plots for trj_analysis generated.")


### 4.2. Radius of Gyration: `rg_analysis`

In [ ]:
rg_result = genesis_exe.rg_analysis(molecule=mol, trajs=traj, 
                                    selection_group=["an:CA"], 
                                    mass_weighted=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(traj.nframe), y=rg_result.rg,
                        mode='lines', name='Rg'))

fig.update_layout(
    title="Rg (C-alpha atoms) vs Time",
    xaxis_title="Frame",
    yaxis_title="Radius of Gyration (Å)",
    showlegend=True
)

fig.show()
print("Plotly plot for rg_analysis generated.")

### 4.3. RMSD `rmsd_analysis`

In [ ]:
rmsd_result = genesis_exe.rmsd_analysis(molecule=mol, trajs=traj, 
                                        selection_group=["an:CA"], 
                                        fitting_method="TR+ROT", 
                                        fitting_atom=1, analysis_atom=1)

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(traj.nframe), y=rmsd_result.rmsd,
                        mode='lines', name='RMSD'))

fig.update_layout(
    title="RMSD (C-alpha, vs. initial) vs Time",
    xaxis_title="Frame",
    yaxis_title="RMSD (Å)",
    showlegend=True
)

fig.show()
print("Plotly plot for rmsd_analysis generated.")

### 4.4. dRMSD `drms_analysis`

### 4.5. Average Structure `avecrd_analysis`

In [ ]:
avecrd_result =  genesis_exe.avecrd_analysis(
                    mol, traj,
                    selection_group = ["segid:PROA and heavy", ],
                    fitting_method = "TR+ROT",
                    fitting_atom = 1,
                    check_only = False,
                    num_iterations = 5,
                    analysis_atom  = 1,
                    )

average_pdb_string = avecrd_result.pdb
print(f"\nAverage structure PDB generated (first 200 chars):\n{average_pdb_string[:200]}...")

In [ ]:
AVERAGE_PDB_PATH_PY3DMOL = pathlib.Path("average_protein_py3dmol.pdb") 
with open(AVERAGE_PDB_PATH_PY3DMOL, "w") as f:
    f.write(average_pdb_string)
    print(f"Average structure saved to {AVERAGE_PDB_PATH_PY3DMOL}")
    view_avg_py3d = py3Dmol.view(width=600, height=400)
    view_avg_py3d.addModel(average_pdb_string, 'pdb');
    view_avg_py3d.setStyle({'cartoon': {'colorscheme': 'ssJmol'}});
    view_avg_py3d.zoomTo()
    print("py3Dmol visualization of average structure (run in Jupyter):")
    view_avg_py3d.show()

### 5.1. Mean Square Displacement (MSD) and Diffusion Coefficient

In [ ]:
msd_result = genesis_exe.msd_analysis(molecule=mol, trajs=traj, 
                                    selection_group=["an:CA"], 
                                    oversample=True, 
                                    delta=traj.nframe -1)

msd_data = msd_result.msd 

print(f"\nMSD data shape: {msd_data.shape}")
time_lags_ps = np.arange(1, msd_data.shape[0] + 1) * 1.0 

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_lags_ps, y=msd_data[:,0],
                        mode='lines', name='MSD'))

fig.update_layout(
    title="MSD vs Time Lag (CA)",
    xaxis_title="Time Lag (ps)",
    yaxis_title="MSD (Å²)",
    showlegend=True
)

fig.show()
print("Plotly plot for MSD generated.")

diffusion_coeffs = genesis_exe.diffusion_analysis(msd_data=msd_data, time_step=1.0, start="20%")

print(f"\nDiffusion Coefficients (Å²/ps) and Intercepts:\n{diffusion_coeffs}")

## 6. Interfacing with External MD Libraries

### 6.1. MDTraj

In [ ]:
print("\nMDTraj imported.")
mdtraj_top = mol.to_mdtraj_topology()
print(f"SMolecule to MDTraj Topology: {mdtraj_top.n_atoms} atoms")
mdtraj_traj_obj = traj.to_mdtraj_trajectory(mol) 
print(f"STrajectories to MDTraj Trajectory: {mdtraj_traj_obj.n_frames} frames, {mdtraj_traj_obj.n_atoms} atoms")
new_straj_mdt, new_smol_mdt = s_trajectories.STrajectories.from_mdtraj_trajectory(mdtraj_traj_obj)
print(f"MDTraj Trajectory to STrajectories: {new_straj_mdt.nframe} frames, {new_straj_mdt.natom} atoms")
new_straj_mdt.free() 

### 6.2 MDAnalysis

In [ ]:
print("\nMDAnalysis imported.")
mda_universe_mol = mol.to_mdanalysis_universe()
print(f"SMolecule to MDAnalysis Universe (topology): {mda_universe_mol.atoms.n_atoms} atoms")
mda_universe_traj_obj = traj.to_mdanalysis_universe(mol) 
print(f"STrajectories to MDAnalysis Universe: {mda_universe_traj_obj.trajectory.n_frames} frames, {mda_universe_traj_obj.atoms.n_atoms} atoms")
#new_straj_mda, new_smol_mda = s_trajectories.STrajectories.from_mdanalysis_universe(mda_universe_traj_obj)
#print(f"MDAnalysis Universe to STrajectories: {new_straj_mda.nframe} frames, {new_straj_mda.natom} atoms")
#new_straj_mda.free()

## 7. Interfacing with Scikit-learn

### 7.1. Feature Preparation

In [ ]:
from sklearn.preprocessing import StandardScaler
print(f"\nScikit-learn imported for feature preparation.")

n_samples = traj.nframe
# Using C-alpha coordinates as features for dimensionality reduction
# Ensure your PDB/PSF allows selection of "name CA"
ca_indices = [i for i, name_array in enumerate(mol.atom_name) if ''.join(name_array).strip() == "CA"]
if not ca_indices: # Fallback if no CA atoms found (e.g. very small dummy molecule)
    print("Warning: No CA atoms found for Scikit-learn features. Using all atoms.")
    features_coords = traj.coords.reshape(n_samples, -1).astype(np.float32)
else:
    features_coords = traj.coords[:, ca_indices, :].reshape(n_samples, -1).astype(np.float32)

print(f"Shape of (CA) coordinate features: {features_coords.shape}")
scaler = StandardScaler()
scaled_features_coords = scaler.fit_transform(features_coords)
print(f"Shape of scaled (CA) coordinate features: {scaled_features_coords.shape}")

### 7.2. t-SNE Dimensionality Reduction

In [ ]:
from sklearn.manifold import TSNE
import plotly.express as px

# Reduce dimensionality to 2 components using t-SNE
# n_samples should be greater than perplexity; adjust perplexity if needed for small N.
perplexity_val = min(30.0, float(n_samples - 1)) if n_samples > 1 else 5.0
if n_samples <= 1 : print("Warning: t-SNE requires multiple samples. Using dummy embedding if n_samples <=1")

tsne = TSNE(n_components=2, random_state=42, perplexity=perplexity_val, max_iter=300, init='pca', learning_rate='auto')
if n_samples > 1 and scaled_features_coords.shape[0] > 0:
    tsne_embedding = tsne.fit_transform(scaled_features_coords)
    print(f"\nt-SNE with Scikit-learn (on CA coordinates):")
    print(f"Shape of t-SNE embedding: {tsne_embedding.shape}")

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=tsne_embedding[:, 0],
        y=tsne_embedding[:, 1],
        mode='markers',
        marker=dict(
            size=5,  # Reduced marker size
            color=np.arange(traj.nframe),
            colorscale='Viridis',  # より時系列の変化がわかりやすいカラースケール
            showscale=True,
            colorbar=dict(title="Time Frame")  # Added colorbar title
        ),
        text=[f'Frame {i}' for i in range(traj.nframe)]
    ))

    fig.update_layout(
        title="t-SNE of Trajectory CA Coordinates",
        xaxis_title="t-SNE Component 1",
        yaxis_title="t-SNE Component 2",
        showlegend=False,
        width=600,  # Adjusted width for a more square aspect ratio
        height=500  # Adjusted height for a more square aspect ratio
    )

    fig.show()
    print("Plotly plot for t-SNE generated.")
else:
    tsne_embedding = np.random.rand(n_samples, 2) # Dummy for plotting if no data
    print("Skipping t-SNE calculation due to insufficient samples or features.")

### 7.3. DBSCAN Clustering

In [ ]:
from sklearn.cluster import HDBSCAN
import plotly.express as px

# Apply HDBSCAN on the t-SNE embedded data or directly on scaled_features_coords
if tsne_embedding is not None and tsne_embedding.shape[0] > 0:
    # Set HDBSCAN parameters
    min_cluster_size = max(5, int(n_samples * 0.05))  # At least 5 samples or 5% of total samples
    min_samples = max(1, int(n_samples * 0.01))  # At least 1 sample or 1% of total samples

    print(f"HDBSCAN parameters: min_cluster_size={min_cluster_size}, min_samples={min_samples}")
    
    hdbscan = HDBSCAN(min_cluster_size=min_cluster_size, min_samples=min_samples)
    cluster_labels_hdbscan = hdbscan.fit_predict(tsne_embedding)
    
    n_clusters_ = len(set(cluster_labels_hdbscan)) - (1 if -1 in cluster_labels_hdbscan else 0)
    n_noise_ = list(cluster_labels_hdbscan).count(-1)

    print(f"\nHDBSCAN Clustering (on t-SNE embedding):")
    print(f"Estimated number of clusters: {n_clusters_}")
    print(f"Estimated number of noise points: {n_noise_}")
    print(f"Cluster labels for first 10 frames: {cluster_labels_hdbscan[:10]}")

    # Create a color map for clusters
    colors = px.colors.qualitative.Set3
    if n_clusters_ > len(colors):
        colors = px.colors.qualitative.Set3 * (n_clusters_ // len(colors) + 1)

    fig = go.Figure()
    
    # Plot each cluster
    for k in range(n_clusters_):
        mask = cluster_labels_hdbscan == k
        cluster_indices = np.where(mask)[0]
        fig.add_trace(go.Scatter(
            x=tsne_embedding[mask, 0],
            y=tsne_embedding[mask, 1],
            mode='markers',
            name=f'Cluster {k}',
            marker=dict(
                size=5,
                color=colors[k % len(colors)]
            ),
            text=[f'Frame {i}, Cluster {k}' for i in cluster_indices],
            hovertemplate='<b>%{text}</b><br>x: %{x:.3f}<br>y: %{y:.3f}<extra></extra>'
        ))

    # Plot noise points if any
    if n_noise_ > 0:
        mask = cluster_labels_hdbscan == -1
        noise_indices = np.where(mask)[0]
        fig.add_trace(go.Scatter(
            x=tsne_embedding[mask, 0],
            y=tsne_embedding[mask, 1],
            mode='markers',
            name='Noise',
            marker=dict(
                size=5,
                color='black'
            ),
            text=[f'Frame {i}, Noise' for i in noise_indices],
            hovertemplate='<b>%{text}</b><br>x: %{x:.3f}<br>y: %{y:.3f}<extra></extra>'
        ))

    fig.update_layout(
        title=f"HDBSCAN Clusters (min_cluster_size={min_cluster_size}, min_samples={min_samples}) on t-SNE",
        xaxis_title="t-SNE Component 1",
        yaxis_title="t-SNE Component 2",
        showlegend=True, 
        width=600,  # Adjusted width for a more square aspect ratio
        height=500  # Adjusted height for a more square aspect ratio
    )

    fig.show()
    print("Plotly plot for HDBSCAN on t-SNE generated.")
else:
    print("\nSkipping HDBSCAN: t-SNE embedding not available.")

## 8. Interfacing with PyTorch: Variational Autoencoder (VAE)

### 8.1. Data Preparation for PyTorch

In [ ]:
import torch
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader
print(f"\nPyTorch version {torch.__version__} imported.")

print(f"\nUsing trajectory with {traj.nframe} frames and {traj.natom} atoms for PyTorch Autoencoder.")
# Use scaled CA coordinates as input for the autoencoder
if 'scaled_features_coords' in globals() and scaled_features_coords is not None:
    pytorch_input_features = torch.tensor(scaled_features_coords, dtype=torch.float32)
    print(f"Shape of PyTorch input tensor (scaled CA coordinates): {pytorch_input_features.shape}")

    # For an autoencoder, the target is the same as the input
    pytorch_target_outputs = pytorch_input_features.clone() 
    print(f"Shape of PyTorch target tensor (same as input): {pytorch_target_outputs.shape}")

    dataset_torch_ae = TensorDataset(pytorch_input_features, pytorch_target_outputs)
    batch_size_torch_ae = min(4, len(dataset_torch_ae)) if len(dataset_torch_ae) > 0 else 1
    dataloader_torch_ae = DataLoader(dataset_torch_ae, batch_size=batch_size_torch_ae, shuffle=True) if len(dataset_torch_ae) > 0 else None
    if dataloader_torch_ae: print(f"Created PyTorch DataLoader for Autoencoder with batch size {batch_size_torch_ae}")
    else: print("Could not create DataLoader for Autoencoder.")
else:
    print("Scaled features not available. Skipping PyTorch Autoencoder data preparation.")
    dataloader_torch_ae = None

### 8.2. Variational Autoencoder

In [ ]:
import torch
from torch import nn
from torch.optim import Adam
from sklearn.datasets import load_digits
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the Variational Autoencoder (VAE)
class VAE(nn.Module):
    def __init__(self, in_dim=30, hidden_dim=128, z_dim=2):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, z_dim * 2)  # mean and variance
        )
        self.decoder = nn.Sequential(
            nn.Linear(z_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, in_dim),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = torch.chunk(h, 2, dim=-1)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

# Initialize the model, optimizer and loss function
model = VAE().to(device)
optimizer = Adam(model.parameters(), lr=1e-4)
criterion = nn.MSELoss(reduction='sum')

# Convert the data to a PyTorch tensor
from sklearn.preprocessing import StandardScaler

# 標準化を実行
scaler = StandardScaler()
normalized_features = scaler.fit_transform(pytorch_input_features.cpu().numpy())
data = torch.tensor(normalized_features, dtype=torch.float32).to(device)

# Training loop
for epoch in range(10000):
    model.train()
    optimizer.zero_grad()
    recon, mu, logvar = model(data)
    loss = criterion(recon, data)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    loss += kl_divergence
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Visualize the 2-D latent space (Plotly)
model.eval()
with torch.no_grad():
    _, mu, _ = model(data)
mu = mu.cpu().numpy()

fig = go.Figure(
    data=go.Scatter(
        x=mu[:, 0],
        y=mu[:, 1],
        mode="markers",
        marker=dict(
            color=np.arange(len(mu)),   # per-point colouring
            colorscale="Viridis",
            showscale=True              # colour bar
        )
    )
)

fig.update_layout(
    width=600, height=500,
    xaxis_title="z_1",
    yaxis_title="z_2"
)

fig.show()
